In [1]:
import sys
sys.path.append('CLIP-dissect') 


In [8]:
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import cv2
import glob
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import matplotlib
import similarity
import utils
import data_utils
%matplotlib inline

In [20]:
from neuron_accuracy import get_actual_labels, neuron_accuracy
from clip_dissect_pipeline import dissect_pipeline

In [10]:
dataset_path = "FER2013"
class_names = os.listdir(dataset_path+"/train")
print(class_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [11]:
emotions = []
for file_name in glob.glob(dataset_path+'/train/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
    emotions.append(emotion)
    
emotions = []
for file_name in glob.glob(dataset_path+'/test/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
    emotions.append(emotion)

In [12]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((48,48))])

train_dataset = ImageFolder(dataset_path+'/train',transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=100*6)
#creating val data loaders
val_dataset = ImageFolder(dataset_path+'/test',transform)
val_loader = DataLoader(dataset=val_dataset,batch_size=100)

In [13]:
# Need to change this part
d_probe = train_dataset
concept_set = class_names
similarity_fn = similarity.soft_wpmi

In [14]:
train_dataset[0][0].shape

torch.Size([3, 48, 48])

In [ ]:
import models
model_path = 'face_emotion_deep_emotion.pth'
target_model = models.Deep_Emotion().to('cuda')
target_model.load_state_dict(torch.load(model_path))

In [16]:
d_probe = 'FER2013'
concept_set = 'CLIP-dissect/data/emotions.txt' # concept set needs to be path to .txt file
similarity_fn = similarity.soft_wpmi

In [ ]:
predicted_emotions, neuron_image_indices = dissect_pipeline(d_probe,concept_set, similarity_fn)

In [ ]:
neuron_accuracy(neuron_image_indices, predicted_emotions)